<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 人类与 AI 消息

In [ ]:
from videos.walkthroughs import walkthrough_31 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，您将了解两种核心聊天消息类型，人类消息和 AI 消息，以及如何在应用代码中明确使用它们。

---

## 目标

在您完成这个 notebook 时，您将会：

- 明确聊天变体 LLM 所使用的基于角色的消息传递系统。
- 学会如何使用 `ChatPromptTemplate` 在提示模板中创建人类和AI 消息。

---

## 导入

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

---

## 创建模型实例

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## 聊天消息

正如我们在课程中之前提到的，与“聊天”或“指令”类型的 LLM 工作时有一些重要的区别，而非聊天模型主要是用来生成接下来应该出现的文本。

我们在整个课程中一直在用 LangChain 构建提示词，主要是通过 `ChatPromptTemplate`。如名称中的“聊天”所暗示，`ChatPromptTemplate` 非常适合用来创建聊天模型需要的提示词，即基于角色的对话交互。

在这个 notebook 的后半部分，我们将学习如何利用我们对聊天模型交互中各种角色的理解。但首先，让我们重温一些熟悉的任务，同时注意提示词和聊天模型的响应是如何被构建以表明它们确实是角色对话的一部分。

---

## 人类消息

首先，我们将创建一个非常简单的聊天提示模板。

In [ ]:
prompt_template = ChatPromptTemplate.from_template("{prompt}")

接下来，我们将通过调用提示模板来实例化一个实际的提示词，然后打印出完整的提示词。

In [ ]:
prompt = prompt_template.invoke({"prompt": "hello"})

In [ ]:
prompt

首先注意到这个提示词是一个 `ChatPromptValue`：这是一个用于聊天模型的提示词。

接下来，我们注意到消息被叫做 `HumanMessage`。在聊天对话中，消息总是与某种角色相关联，而这条消息被理解为是由“人类”角色生成的。

调用提示词的 `to_messages` 方法能更清晰地看到。

In [ ]:
prompt.to_messages()

---

## AI 消息

让我们创建一个非常基本的链，以便我们可以将提示词发送给聊天模型，然后仔细查看它返回给我们的消息。值得注意的是，我们在链的末尾没有包含解析器，因为我们想要探索模型的整个响应。

In [ ]:
chain = prompt_template | llm

In [ ]:
response = chain.invoke({"prompt": "hello"})

In [ ]:
response

模型的响应是一个 `AIMessage`，我们（以及模型）可以理解为这个消息是由“AI”角色生成的。

---

## 在提示词中明确使用角色

在 LangChain 的底层，`ChatPromptTemplate` 一直在管理“人类”提示词和“AI”聊天模型响应的角色。不过，LangChain 也提供了简单易用的机制来进行明确的角色管理。

最简单的方法是用 `ChatPromptTemplate.from_messages`，它接受一个消息列表，其中每条消息是一个二元组，第一个值表示与消息相关联的角色，第二个值是消息的内容。

我们来用 `ChatPromptTemplate.from_messages` 重新创建之前 `from_template` 创建的完全相同的提示词。这次我们明确声明提示将与“人类”角色相关联。

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("human", "{prompt}")
])

可以看到，就像之前做的那样，提示是一个 `ChatPromptValue`，其中包含一个 `HumanMessage` 类型的消息。

In [ ]:
prompt = prompt_template.invoke({"prompt": "hello"})

In [ ]:
prompt

In [ ]:
prompt.to_messages()

我们可以像使用 `from_template` 创建的提示词那样使用这个提示模板。

In [ ]:
chain = prompt_template | llm

In [ ]:
response = chain.invoke({"prompt": "hello"})

In [ ]:
response

---

## 直接使用 ChatPromptMessages

顺便提一下，在 LangChain 的最新版本中，我们可以直接使用 `ChatPromptMessages`，这相当于使用 `ChatPromptTemplate.from_messages`。因此下面这两种写法是等价的。

In [ ]:
ChatPromptTemplate.from_messages([
    ("human", "{prompt}")
])

In [ ]:
ChatPromptTemplate([
    ("human", "{prompt}")
])

本课程中，我们主要用 `from_messages`，因为您在文档和文献中会更频繁地见到这种用法，不过您可以随意使用这两种方法中的任何一种。

---

## 练习：创建一个明确的人类消息

作为一个非常简单的练习，为了让您主动使用人类消息，请重构以下链以使用 `ChatPromptTemplate.from_messages`。

In [ ]:
template = ChatPromptTemplate.from_template("Give the concise etomology of the following English word: {word}")

In [ ]:
parser = StrOutputParser()

In [ ]:
chain = template | llm | parser

In [ ]:
print(chain.invoke({"word": "learning"}))

### 您的代码

### 参考答案

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("human", "Give the concise etomology of the following English word: {word}")
])

In [ ]:
parser = StrOutputParser()

In [ ]:
chain = template | llm | parser

In [ ]:
print(chain.invoke({"word": "learning"}))

---

## 明确使用 AI 角色

除了提供“人类”角色消息外，我们还可以将传递 AI 角色的消息给模型。

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("human", "Hello."),
    ("ai", "Hello, how are you?"),
    ("human", "{prompt}")
])

如果我们调用这个提示，可以看到与我们之前发送给聊天模型的提示不同，它包含了 3 条消息，其中两条与人类角色相关，另一条与 AI 角色相关。

In [ ]:
prompt = prompt_template.invoke({"prompt": "I'm well, thanks!"})

In [ ]:
prompt.to_messages()

这种能力使我们能够构建包含额外上下文的提示，以便模型在生成响应时使用。

从模型的角度来看，它看到的是当前聊天对话中已经发生的内容，这些关于已经发生的上下文可以影响模型在后续对话轮次中的响应。

有两种主要的利用这种能力的方式。

第一种是当我们想要实现聊天机器人功能时。在每次人类与 AI 交互后，可以将这次交互添加到我们的提示词中。因此，每当我们向聊天机器人发送消息时，它都能了解到完整对话上下文，从而能更恰当地做出回应。稍后我们将详细探讨如何创建聊天机器人功能。

第二种是构建我们自己虚构的人类与 AI 的交互，作为模型如何应对后续人类消息的示例。这种提供人类与 AI 交互示例的技术被称为少样本提示（few-shot prompting），我们将在下一个 notebook 中讨论这个话题。

---

## 使用 `HumanMessage` 和 `AIMessage`

在 LangChain 中，通常有多种方式来完成同样的事，这在创建角色特定的消息时也适用。目前为止，我们一直在用二元组语法创建与角色明确相关的消息。

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("human", "Hello."),
    ("ai", "Hello, how are you?"),
    ("human", "{prompt}")
])

实际上还可以用 LangChain 的 `HumanMessage` 和 `AIMessage` 类。

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

下面这种写法跟二元组实现是等价的。

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    HumanMessage(content="Hello"),
    AIMessage(content="Hello, how are you?"),
    HumanMessage(content="{prompt}")
])

这真的只是一个选择问题，您可以在应用中选择任何一种写法，这个课程中也可以随意选择。重要的是能够识别和理解它们，因为您可能会在技术文档和示例中看到这两种。

## 总结

现在您已经熟悉了人类和 AI 消息，包括如何在聊天提示模板中编写它们。下一个 notebook 您将学习一种强大且流行的技术，称为少样本提示，这将利用您的消息编写技能为聊天模型提供示例，从而影响它们的行为。